In [1]:
import pickle
import time
from typing import List

import numpy as np
from tqdm import tqdm

from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis, get_pauli_basis, get_comp_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_x0_1q,  get_x1_1q, get_y0_1q, get_y1_1q, get_z0_1q, get_z1_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def estimate(name: str,
             true_object: Povm,
             states: List[State],
             measurement_n: int,
             num_data: List[int], iterations: int, on_para_eq_constraint: bool=True):
    povmt = StandardPovmt(states, measurement_n, on_para_eq_constraint=on_para_eq_constraint)

    # generate empi dists and calc estimate
    obj_sequences = []
    start = time.time()
    for iteration in range(iterations):
        seeds = [iteration] * len(num_data)
        empi_dists_seq = povmt.generate_empi_dists_sequence(
            true_object, num_data, seeds
        )

        estimator = LinearEstimator()
        obj_sequence = estimator.calc_estimate_sequence(povmt, empi_dists_seq)

        info = {
            "iteration": iteration,
            "empi_dists_seq": empi_dists_seq,
            "obj_sequence": obj_sequence.estimated_var_sequence,
        }
        print(info)
        obj_sequences.append(obj_sequence.estimated_qoperation_sequence)

    end = time.time()
    print(f"time(s)={end - start}")
    #with open(f"state_obj_sequences_{name}_{on_para_eq_constraint}_{iterations}.pkl", "wb") as f:
    #    pickle.dump(obj_sequences, f)

    #with open("obj_sequences.pkl", "rb") as f:
    #    obj_sequences = pickle.load(f)
    return obj_sequences

In [4]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

# |+><+|
state_x0 = get_x0_1q(c_sys)
# |+i><+i|
state_y0 = get_y0_1q(c_sys)
# |0><0|
state_z0 = get_z0_1q(c_sys)
# |1><1|
state_z1 = get_z1_1q(c_sys)
states = [state_x0, state_y0, state_z0, state_z1]

In [5]:
# parameter
num_data = [100, 1000, 10000, 100000]
iterations = 10
true_object = get_x_measurement(c_sys)  # TODO: 真値で置き換える
measurement_n = 2  # 測定値の数

param_affine_est_linear = estimate("name", true_object, states, measurement_n, num_data, iterations, on_para_eq_constraint=True)
param_linear_est_linear = estimate("name", true_object, states, measurement_n, num_data, iterations, on_para_eq_constraint=False)

{'iteration': 0, 'empi_dists_seq': [[(100, array([1., 0.])), (100, array([0.51, 0.49])), (100, array([0.51, 0.49])), (100, array([0.51, 0.49]))], [(1000, array([1., 0.])), (1000, array([0.517, 0.483])), (1000, array([0.517, 0.483])), (1000, array([0.517, 0.483]))], [(10000, array([1., 0.])), (10000, array([0.5064, 0.4936])), (10000, array([0.5064, 0.4936])), (10000, array([0.5064, 0.4936]))], [(100000, array([1., 0.])), (100000, array([0.50055, 0.49945])), (100000, array([0.50055, 0.49945])), (100000, array([0.50055, 0.49945]))]], 'obj_sequence': [array([7.21248917e-01, 6.92964646e-01, 7.77156117e-16, 0.00000000e+00]), array([7.31148412e-01, 6.83065151e-01, 8.32667268e-16, 0.00000000e+00]), array([7.16157748e-01, 6.98055814e-01, 7.77156117e-16, 0.00000000e+00]), array([7.07884599e-01, 7.06328964e-01, 7.77156117e-16, 0.00000000e+00])]}
{'iteration': 1, 'empi_dists_seq': [[(100, array([1., 0.])), (100, array([0.51, 0.49])), (100, array([0.51, 0.49])), (100, array([0.51, 0.49]))], [(1000,

{'iteration': 0, 'empi_dists_seq': [[(100, array([1., 0.])), (100, array([0.51, 0.49])), (100, array([0.51, 0.49])), (100, array([0.51, 0.49]))], [(1000, array([1., 0.])), (1000, array([0.517, 0.483])), (1000, array([0.517, 0.483])), (1000, array([0.517, 0.483]))], [(10000, array([1., 0.])), (10000, array([0.5064, 0.4936])), (10000, array([0.5064, 0.4936])), (10000, array([0.5064, 0.4936]))], [(100000, array([1., 0.])), (100000, array([0.50055, 0.49945])), (100000, array([0.50055, 0.49945])), (100000, array([0.50055, 0.49945]))]], 'obj_sequence': [array([ 7.21248917e-01,  6.92964646e-01, -3.33066907e-16,  0.00000000e+00,
        6.92964646e-01, -6.92964646e-01, -3.33066907e-16,  0.00000000e+00]), array([ 7.31148412e-01,  6.83065151e-01, -4.44089210e-16,  0.00000000e+00,
        6.83065151e-01, -6.83065151e-01, -2.22044605e-16,  0.00000000e+00]), array([ 7.16157748e-01,  6.98055814e-01, -3.33066907e-16,  0.00000000e+00,
        6.98055814e-01, -6.98055814e-01, -3.33066907e-16,  0.000000

{'iteration': 8, 'empi_dists_seq': [[(100, array([1., 0.])), (100, array([0.48, 0.52])), (100, array([0.48, 0.52])), (100, array([0.48, 0.52]))], [(1000, array([1., 0.])), (1000, array([0.49, 0.51])), (1000, array([0.49, 0.51])), (1000, array([0.49, 0.51]))], [(10000, array([1., 0.])), (10000, array([0.5034, 0.4966])), (10000, array([0.5034, 0.4966])), (10000, array([0.5034, 0.4966]))], [(100000, array([1., 0.])), (100000, array([0.50337, 0.49663])), (100000, array([0.50337, 0.49663])), (100000, array([0.50337, 0.49663]))]], 'obj_sequence': [array([ 6.78822510e-01,  7.35391052e-01, -3.33066907e-16,  0.00000000e+00,
        7.35391052e-01, -7.35391052e-01, -3.33066907e-16,  0.00000000e+00]), array([ 6.92964646e-01,  7.21248917e-01, -3.33066907e-16,  0.00000000e+00,
        7.21248917e-01, -7.21248917e-01, -3.33066907e-16,  0.00000000e+00]), array([ 7.11915107e-01,  7.02298455e-01, -3.33066907e-16,  0.00000000e+00,
        7.02298455e-01, -7.02298455e-01, -3.33066907e-16,  0.00000000e+00